# 세션 2 – RAG 평가와 ragas

ragas 메트릭을 사용하여 최소 RAG 파이프라인 평가: answer_relevancy, faithfulness, context_precision.


# 시나리오
이 시나리오는 최소한의 Retrieval Augmented Generation (RAG) 파이프라인을 로컬에서 평가합니다. 우리는 다음을 수행합니다:
- 작은 합성 문서 코퍼스를 정의합니다.
- 문서를 임베딩하고 단순한 유사성 검색기를 구현합니다.
- 로컬 모델(Foundry Local / OpenAI 호환)을 사용하여 근거 있는 답변을 생성합니다.
- ragas 메트릭(`answer_relevancy`, `faithfulness`, `context_precision`)을 계산합니다.
- 빠른 반복을 위해 답변 관련성만 계산하는 FAST 모드(`RAG_FAST=1` 환경 변수)를 지원합니다.

이 노트북을 사용하여 로컬 모델과 임베딩 스택이 사실에 근거한 답변을 생성하는지 확인한 후 더 큰 코퍼스로 확장하세요.


### 설명: 의존성 설치
필요한 라이브러리 설치:
- 로컬 모델 관리를 위한 `foundry-local-sdk`.
- `openai` 클라이언트 인터페이스.
- 밀집 임베딩을 위한 `sentence-transformers`.
- 평가 및 메트릭 계산을 위한 `ragas` + `datasets`.
- ragas LLM 인터페이스를 위한 `langchain-openai` 어댑터.

재실행해도 안전하며, 환경이 이미 준비된 경우 생략 가능합니다.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### 설명: 핵심 임포트 및 메트릭
핵심 라이브러리와 ragas 메트릭을 로드합니다. 주요 구성 요소:
- 임베딩을 위한 SentenceTransformer.
- `evaluate`와 선택된 ragas 메트릭.
- 평가 코퍼스를 구성하기 위한 `Dataset`.
이러한 임포트는 원격 호출을 유발하지 않습니다 (임베딩을 위한 모델 캐시 로드 가능성을 제외하고).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### 설명: 간단한 코퍼스 및 QA 정답 데이터
미니어처 메모리 내 코퍼스(`DOCS`), 사용자 질문 세트, 그리고 예상 정답 데이터를 정의합니다. 이를 통해 외부 데이터 가져오기 없이 빠르고 결정적인 메트릭 계산이 가능합니다. 실제 상황에서는 운영 환경의 쿼리와 신중히 선정된 답변을 샘플링하게 됩니다.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### 설명: 서비스 초기화, 임베딩 및 안전 패치
Foundry Local 매니저를 초기화하고, `promptTemplate`에 대한 스키마 드리프트 안전 패치를 적용하며, 모델 ID를 확인하고, OpenAI 호환 클라이언트를 생성한 뒤, 문서 코퍼스에 대한 밀집 임베딩을 사전 계산합니다. 이는 검색 및 생성에 재사용 가능한 상태를 설정합니다.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### 설명: Retriever 함수
정규화된 임베딩을 사용하여 내적(dot product)을 기반으로 한 단순 벡터 유사성 검색기를 정의합니다. 기본적으로 상위 k개의 문서(k=2)를 반환합니다. 실제 환경에서는 확장성과 지연 시간을 위해 ANN 인덱스(FAISS, Chroma, Milvus)로 교체하세요.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### 설명: 생성 함수
`generate`는 제한된 프롬프트를 구성하며 (시스템이 오직 컨텍스트만 사용하도록 지시) 로컬 모델을 호출합니다. 낮은 온도(0.1)는 창의성보다 정확한 추출을 선호합니다. 잘라낸 답변 텍스트를 반환합니다.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### 설명: Fallback 클라이언트 초기화
이전 초기화 셀이 건너뛰어지거나 실패했을 경우에도 `client`가 존재하도록 보장하여 이후 평가 단계에서 NameError를 방지합니다.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### 설명: 평가 루프 및 메트릭
평가 데이터셋을 생성합니다 (필수 열: question, answer, contexts, ground_truths, reference) 그런 다음 선택된 ragas 메트릭을 반복적으로 실행합니다.

최적화:
- FAST_MODE는 빠른 스모크 테스트를 위해 답변 관련성으로 제한합니다.
- 메트릭별 루프는 하나의 메트릭이 실패할 경우 전체 재계산을 방지합니다.

메트릭 -> 점수 (실패 시 NaN)의 딕셔너리를 출력합니다.


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
